# (OpenCV - Chap7) 영역처리
> 영역처리
- toc: true
- branch: master
- badges: false
- comments: true
- author: dinonene
- categories: [python]

## 7.1.2 블러링
블러링은 영상에서 화소값이 급격하게 변하는 부분들을 감소시켜 점진적으로 변하게 함으로써 영상에서 전체적으로 부드러운 느낌이 나게 하는 기술이다. 교재에 따라서 스무딩(smoothing)이라 하기도 한다.

In [5]:
((60+90+90+200+100+100)+300)/9

104.44444444444444

In [2]:
import numpy as np, cv2

In [3]:
# 회선수행 함수 - 행렬 처리 방식 (속도 면에서 유리)
def filter(image, mask):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.float32) # 회선 결과 저장 행렬
    
    ycenter, xcenter = rows//2, cols//2 # 마스크 중심 좌표
    
    for i in range(ycenter, rows - ycenter): # 입력 행렬 반복 순회
        for j in range(xcenter, cols - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1 # 관심 영역 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1 # 관심 영역 너비 범위
            roi = image[y1:y2, x1:x2].astype('float32') # 관심 영역 형변환
            tmp = cv2.multiply(roi, mask) # 회선 적용 - 원소 간 곱셈
            dst[i, j] = cv2.sumElems(tmp)[0] # 출력 화소 저장
    return dst # 자료형 변환하여 반환

In [4]:
# 회선 수행 함수 - 화소 직접 근접
def filter2(image, mask):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.float32)                 # 회선 결과 저장 행렬
    xcenter, ycenter = mask.shape[1]//2, mask.shape[0]//2  # 마스크 중심 좌표

    for i in range(ycenter, rows - ycenter):                  # 입력 행렬 반복 순회
        for j in range(xcenter, cols - xcenter):
            sum = 0.0
            for u in range(mask.shape[0]):                    # 마스크 원소 순회
                for v in range(mask.shape[1]):
                    y, x = i + u - ycenter , j + v - xcenter
                    sum += image[y, x] * mask[u, v]           # 회선 수식
            dst[i, j] = sum
    return dst

In [5]:
image = cv2.imread("./ghtop_images/chap07_images/filter_blur.jpg", cv2.IMREAD_GRAYSCALE)  # 영상 읽기
if image is None: raise Exception("영상파일 읽기 오류")

In [6]:
# 블러링 마스크 원소 지정     
data = [1/9, 1/9, 1/9,
        1/9, 1/9, 1/9,
        1/9, 1/9, 1/9]

# 마스크 행렬 생성
mask = np.array(data, np.float32).reshape(3, 3)
blur1 = filter(image, mask)                # 회선 수행 - 행렬 처리 방식
blur2 = filter2(image, mask)               # 회선 수행 - 화소 직접 접근
blur1 = blur1.astype('uint8')              # 행렬 표시위해 uint8형 변환
blur2 = cv2.convertScaleAbs(blur2)

In [7]:
cv2.imshow('image', image)
cv2.imshow('blur1', blur1)
cv2.imshow('blur2', blur2)

cv2.waitKey(0)
cv2.destroyAllWindows() # 모든 창 닫기

## Summary

`1.` **회선(convolution)** 은 마스크 내의 원소값과 공간 영역에 있는 입력 영상의 화소값들을 대응되게 곱하여 출력 화소값을 계산하는 것을 말한다. 이때, 입력 영상에 곱해지는 이 마스크를 커널(kernel), 윈도우(window), 필터(filter)등의 용어로 부른다.

`2.` **블러링(bluring)** 은 회선 마스크의 원소를 모두 같은 값으로 지정해 수행하며, 전체 합이 1이 되어야 한다. 출력 영상에서 이웃하는 화소들이 비슷한 값을 갖기 때문에 부드러운 영상이 되며, 흐려지는 결과가 발생한다.

`3.` **샤프닝(sharpening)** 은 회선 마스크에서 중심 계수와 주변 계수의 차이를 크게 만들어 출력 화소가 도드라지게 함으로써 선명하고 날카로운 영상을 만드는 방법이다. 중심계수는 아주 큰 양수값을 갖게 하며, 주변 계수는 음수값을 갖게 해서 전체 합이 1이되게 한다.

`4.` 영상처리에서 **에지**는 **"화소값이 급격하게 변화하는 부분"** 으로 정의한다. 이것은 객체에서 크기, 위치, 모양을 인지할 수 있으며, 그 뱡향성을 탐지할 수 있다.

`5.` 에지는 이웃하는 두 화소의 차분으로 구할 수 있으며, 이것은 미분 공식과 유사하다. 따라서 미분 마스크로 회선을 수행하면 에지를 검출할 수 있다. 이것은 1차 미분 마스크라고 하며, 대표적으로 로버츠(Roberts), 소벨(Sobel), 프리윗(Prewitt) 등이 있다.

`6.` 1차 미분 연산자는 점진적으로 변화하는 부분까지 민감하게 에지를 검출하여 너무 많은 에지가 나타날 수 있다. 이를 보완하는 방법으로 1차 미분한 결과에 미분을 한 번 더 하는 방법인 2차 미분 연산이 잇다. 대표적으로 라플라시안, LoG(Laplacian of Gaussian), DoG(Difference of Gaussian)등의 방법이 있다.

`12.` 가우시안 블러링은 정규분포 곡선을 갖는 마스크를 가우시안 수식에 따라서 생성하고 이 마스크로 회선을 수행하는 방법이다. 표준편차로 정규분포 마스크를 생성할 수 있다. 표준편차가 클수록 많이 흐려진 영상을 생성한다.

`13.` 모폴로지는 행태학적 방법을 여상처리에 적용한 것으로서 침식과 팽창 연산이 있다. 
- 침식 연산은 객체의 크기가 축소되기 때문에 영상 내에 존재하는 작은 크기의 잡음을 제거하는데에 효과적이다.
- 팽창연산은 객체의 크기가 확대되어 객체 내부의 빈 공간을 메우는 역할을 한다.

`14.` 열림 연산은 침식 연산 수행 후에 팽창 연산을 수행한다. 침식 연산으로 객체는 축소되고, 배경의 잡음들은 제거되며, 팽창 연산으로 축소되었던 객체들이 원래 크기로 돌아간다.

`15.` 닫힘 연산은 팽창 연산 수행 후에 침식 연산을 수행한다. 팽창 연산으로 객체가 확장되어 객체 내부의 빈 공간이 매워진다. 다음으로 침식 연산으로 확장되었던 객체의 크기가 원래대로 축소된다.

ref : https://pinkocto.github.io/BP2022/python/2022/10/15/opencv.html